Medium - Length of the longest alphabetical continuous substring

In [ ]:
class Solution:
    def longestContinuousSubstring(self, s: str) -> int:
        #sliding window approach
        left = 0
        ans = 0
        for rightIndex, val in enumerate(s):
            if ord(val) - ord(s[rightIndex-1 if rightIndex>0 else 0]) != 1:
                left = rightIndex
            ans = max(ans, rightIndex-left+1)
        return ans

Medium: Nested List Weight Sum

You are given a nested list of integers nestedList. Each element is either an integer or a list whose elements may also be integers or other lists.
The depth of an integer is the number of lists that it is inside of. For example, the nested list [1,[2,2],[[3],2],1] has each integer's value set to its depth.
Return the sum of each integer in nestedList multiplied by its depth.

Input: nestedList = [[1,1],2,[1,1]]
Output: 10
Explanation: Four 1's at depth 2, one 2 at depth 1. 1*2 + 1*2 + 2*1 + 1*2 + 1*2 = 10.

To make it simplier, we have 3 premade functions we can call, nestedList.isInteger(), nestedList.getInteger(), nestedList.getList()

In [ ]:
def depthSum(self, nestedList: List[NestedInteger]) -> int:
    def helper(curr_list, curr_depth):
        ans = 0
        for item in curr_list:
            if item.isInteger():
                ans += item.getInteger()*curr_depth
            else:
                ans += helper(item.getList(), curr_depth+1)
        return ans
    return helper(nestedList, 1)

Easy - Largest Subarray Length K

An array A is larger than some array B if for the first index i where A[i] != B[i], A[i] > B[i].

For example, consider 0-indexing:

[1,3,2,4] > [1,2,2,4], since at index 1, 3 > 2.
[1,4,4,4] < [2,1,1,1], since at index 0, 1 < 2.
A subarray is a contiguous subsequence of the array.

Given an integer array nums of distinct integers, return the largest subarray of nums of length k.

In [ ]:
#Greedy solution, since each number is unique we can use it
class Solution:
    def largestSubarray(self, nums: List[int], k: int) -> List[int]:
        n = len(nums)
        ans_index = 0
        for i in range(n-k+1):
            if nums[i] > nums[ans_index]:
                ans_index = i
        return nums[ans_index: ans_index+k]


Medium - Length of the longest alphabetical continuous substring